# HELPERS


In [0]:
# pytorch
import torch
import numpy as np
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
from IPython.display import clear_output

def to_gpu(x):
  if torch.cuda.is_available():
    x = x.cuda()
  return x

# Training GANS


## Architecture

![](https://sthalles.github.io/assets/dcgan/GANs.png)

## GAN seen as a *minmax* game

![](https://drive.google.com/uc?export=view&id=1KVkTMjTT7jr8gfIv-1c4LRosrCZRpJd9)

## Training intuition

![](https://drive.google.com/uc?export=view&id=15G9ZuSZqLG-E8LufyT-UnN9V0lme3nJZ)

## Training procedure

![alt text](https://drive.google.com/uc?export=view&id=1_JMKgK48QbhU9W9F9DG6dPQpYZPcYGSE)


In [0]:
def gan_magic(generator, discriminator, data_loader, noise, plot, epochs=100,
              disc_steps=1, test_samples=10, draw_every_n=1, d_lr=0.001, g_lr=0.001):
  # Total number of epochs to train
  num_epochs = epochs

  test_noise = noise(test_samples)
  
  d_optimizer = optim.Adam(discriminator.parameters(), lr=d_lr)
  g_optimizer = optim.Adam(generator.parameters(), lr=g_lr)
  
  loss = nn.BCELoss()
  
  def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)

    # Calculate error and backpropagate
    error_real = loss(prediction_real, ones_target(N) )
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, zeros_target(N))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error and predictions for real and fake inputs
    return error_real + error_fake, prediction_real, prediction_fake
  
  def train_generator(optimizer, fake_data):
    N = fake_data.size(0)

    # Reset gradients
    optimizer.zero_grad()

    # Sample noise and generate fake data
    prediction = discriminator(fake_data)

    # Calculate error and backpropagate
    error = loss(prediction, ones_target(N))
    error.backward()

    # Update weights with gradients
    optimizer.step()

    # Return error
    return error
  


  for epoch in range(num_epochs):
      for n_batch, (real_batch,_) in enumerate(data_loader):
          N = real_batch.size(0)
          
          real_data = to_gpu(Variable(real_batch))

          # 1. Train Discriminator
          # 1.1 Generate fake data and detach 
          # (so gradients are not calculated for generator)
          fake_data = generator(noise(N)).detach()
          # 1.2 Train D
          d_error, d_pred_real, d_pred_fake = \
              train_discriminator(d_optimizer, real_data, fake_data)
      
          # 2. Train Generator every "k = disc_steps" steps
          if n_batch % disc_steps == 0:        
            # 2.1 Generate fake data again
            fake_data = generator(noise(N))
            # 2.2 Train G
            g_error = train_generator(g_optimizer, fake_data)

      # Display samples every few batches
      if epoch % draw_every_n == 0: 
          test_images = generator(test_noise).cpu()
          test_images = test_images.data
          plot(test_images)
          
      # Display status Logs
      display_status(
          epoch, num_epochs,
          d_error, g_error, d_pred_real, d_pred_fake
      )
        
 

In [0]:
def display_status(epoch, num_epochs, d_error, g_error, d_pred_real, d_pred_fake):

    # var_class = torch.autograd.variable.Variable
    if isinstance(d_error, torch.autograd.Variable):
        d_error = d_error.data.cpu().numpy()
    if isinstance(g_error, torch.autograd.Variable):
        g_error = g_error.data.cpu().numpy()
    if isinstance(d_pred_real, torch.autograd.Variable):
        d_pred_real = d_pred_real.data
    if isinstance(d_pred_fake, torch.autograd.Variable):
        d_pred_fake = d_pred_fake.data


    print('Epoch: [{}/{}]'.format(epoch,num_epochs))
    print('Discriminator Loss: {:.4f}, Generator Loss: {:.4f}'.format(d_error, g_error))
    print('D(x): {:.4f}, D(G(z)): {:.4f}'.format(d_pred_real.mean(), d_pred_fake.mean()))


In [0]:
def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    return to_gpu(data)
def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    return to_gpu(data)

# Vanilla GAN

In [0]:
LATENT_SIZE = 10
BATCH_SIZE = 32
TEST_SAMPLES = 16

def circle_noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.rand(size, LATENT_SIZE))
    return to_gpu(n)

In [0]:
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt
import random

# generate 2d classification dataset
X, _ = make_circles(n_samples=400, noise=0.05, factor=0.99)

def plot_circle(data):
  xax = data[:,0]
  yax = data[:,1]

  plt.scatter(xax, yax, c='red')
  plt.show()

class CircleGenerator:
  def __init__(self, data):
    self.data = data
    
    
  def gen_batches(self):
    for _ in range(len(self.data) // BATCH_SIZE):
      dcopy = self.data.copy()
      np.random.shuffle(dcopy)
      yield torch.from_numpy(dcopy[:BATCH_SIZE].astype(np.float32)), None # second one just for compatibility

  def __iter__(self):
    return self.gen_batches()

  
circle_data_loader = CircleGenerator(X)

plot_circle(X)

In [0]:
class DiscriminatorNet(torch.nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = 2
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 50),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(50, 20),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(20, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.out(x)
        return x

discriminator = to_gpu(DiscriminatorNet())


class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 10
        n_out = 2
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 20),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(            
            nn.Linear(20, 50),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(50, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.out(x)
        return x

generator = to_gpu(GeneratorNet())

In [0]:
gan_magic(generator, discriminator, circle_data_loader, circle_noise,
          plot_circle, epochs=1000, disc_steps=10, test_samples=200, draw_every_n=10, d_lr=0.01, g_lr=0.01)

# DCGAN

In [0]:
LATENT_SIZE = 100
GEN_FEATS = 32
DIS_FEATS = 32
NUM_CHANNELS = 1
IMAGE_W = 28
IMAGE_H = 28
BATCH_SIZE = 32
TEST_SAMPLES = 16

In [16]:
#@title Convolutions size check (square images)

in_channels = 0  #@param {type: "slider", min: 0, max: 100}
out_channels = 28  #@param {type: "slider", min: 0, max: 100}
image_size = 1  #@param {type: "slider", min: 0, max: 100}
kernel_size = 4  #@param {type: "slider", min: 1, max: 10}
stride = 1  #@param {type: "slider", min: 0, max: 10}
padding = 0  #@param {type: "slider", min: 0, max: 10}


transpose_out = (image_size-1) * stride - (2*padding) + kernel_size
conv_out = (image_size + (2 * padding) - (kernel_size -1) -1) / stride + 1


print("ConvTranspose shape:", ('BS', out_channels, transpose_out, transpose_out))
print("ConvTranspose shape:", ('BS', out_channels, conv_out, conv_out))

ConvTranspose shape: ('BS', 28, 4, 4)
ConvTranspose shape: ('BS', 28, -2.0, -2.0)


In [0]:
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize([.5], [.5])
        ])
    out_dir = './dataset'
    return datasets.FashionMNIST(root=out_dir, train=True, transform=compose, download=True)

# Load data
data = mnist_data()
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=BATCH_SIZE, shuffle=True)
# Num batches
num_batches = len(data_loader)

In [0]:
def plot_images(images):
    images = images.numpy().reshape(images.shape[0], 28, 28)
    
    images_row = 6
    num_images = len(images)

    
    f, axarr = plt.subplots((num_images // images_row) + 1, 
                            images_row)
  
    for ax in axarr.flatten():
      ax.axis('off')
  
    for i, image in enumerate(images):
      ax = axarr[i // images_row, i % images_row]
      ax.imshow(image)
    clear_output()
    plt.show()

In [0]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [0]:
class DiscriminatorNet(torch.nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = 784
        n_out = 1
        nc = 1
        ndf = 64
 
        self.conv1 = nn.Sequential(
            # input is (nc) x 28 x 28
            nn.Conv2d(NUM_CHANNELS, DIS_FEATS, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.conv2 = nn.Sequential(
            # state size. (ndf) x 14 x 14
            nn.Conv2d(DIS_FEATS, DIS_FEATS*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(DIS_FEATS*2),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.conv3 = nn.Sequential(
            # state size. (ndf*2) x 7 x 7
            nn.Conv2d(DIS_FEATS*2, DIS_FEATS*4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(DIS_FEATS*4),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.linear1 = nn.Sequential(   
            # state size. (ndf*4) x 4 x 4
            nn.Linear(DIS_FEATS*4 * 4 * 4, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, DIS_FEATS*4*4*4)
        x = self.linear1(x)
        return x

discriminator = to_gpu(DiscriminatorNet())
discriminator.apply(weights_init)

In [0]:

class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
       
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(LATENT_SIZE, GEN_FEATS*4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(GEN_FEATS*4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(GEN_FEATS*4, GEN_FEATS*2, 3, 2, 1, bias=False),
            nn.BatchNorm2d(GEN_FEATS*2),
            nn.ReLU(True),
            # state size. (ngf*4) x 7 x 7
            nn.ConvTranspose2d(GEN_FEATS*2,GEN_FEATS, 4, 2, 1, bias=False),
            nn.BatchNorm2d(GEN_FEATS),
            nn.ReLU(True),
            # state size. (ngf*2) x 14 x 14
            nn.ConvTranspose2d(GEN_FEATS, NUM_CHANNELS, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (NUM_CHANNELS x 28 x 28)
        )

    def forward(self, x):
        x = self.main(x)
        return x

generator = to_gpu(GeneratorNet())
generator.apply(weights_init)

In [0]:
def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, LATENT_SIZE, 1, 1))
    return to_gpu(n)
  

In [0]:
gan_magic(generator, discriminator, data_loader, noise, plot_images, test_samples=TEST_SAMPLES,
          disc_steps=1, d_lr=0.0001, g_lr=0.0001)